In [1]:
import numpy as np
import pandas as pd
import sqlite3
import sqlalchemy as sa

In [4]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('/home/john/Documents/MCXrayTransport/data/data_sources/EPDL/EPDL.db')

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Update the density values in Materials based on the matching symbols in Elements
query = """
UPDATE Materials
SET Density = Elements.Density
FROM Elements
WHERE Materials.Name = Elements.Symbol;
"""

cursor.execute(query)

# Commit the changes
conn.commit()

# Close the connection
conn.close()

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_table_data(url):
    response = requests.get(url)
    html_content = response.text

    soup = BeautifulSoup(html_content, "html.parser")
    table = soup.find_all("table")[0]
    rows = table.find_all("tr")[3:]

    data = []
    for row in rows:
        cols = row.find_all("td")
        cols = [col.text.strip() for col in cols if col.text.strip()]
        if cols and len(cols) == 5:
            # Formatting Composition as Dictionary
            composition_str = cols[4].split('\n')
            composition = {}
            for s in composition_str:
                key, val = s.split(':')
                composition[int(key)] = float(val)

            cols[4] = composition
            data.append(cols)

    df = pd.DataFrame(data, columns=['Material', 'Z/A', 'I', 'Density', 'Composition'])
    df['Z/A'] = df['Z/A'].astype(float)
    df['I'] = df['I'].astype(float)
    df['Density'] = df['Density'].astype(float)
    return df


In [8]:
url = "https://physics.nist.gov/PhysRefData/XrayMassCoef/tab2.html"
df = extract_table_data(url)
df.head()

,Material,Z/A,I,Density,Composition
0,A-150 Tissue-Equivalent Plastic,0.54903,65.1,1.127000,"{1: 0.10133, 6: 0.775498, 7: 0.035057, 8: 0.05..."
1,Adipose Tissue (ICRU-44),0.55579,64.8,0.950000,"{1: 0.114, 6: 0.598, 7: 0.007, 8: 0.278, 11: 0..."
2,"Air, Dry (near sea level)",0.49919,85.7,0.001205,"{6: 0.000124, 7: 0.755268, 8: 0.231781, 18: 0...."
3,Alanine,0.53876,71.9,1.424000,"{1: 0.079192, 6: 0.404437, 7: 0.157213, 8: 0.3..."
4,B-100 Bone-Equivalent Plastic,0.52740,85.9,1.450000,"{1: 0.065473, 6: 0.536942, 7: 0.0215, 8: 0.032..."


In [9]:
def fill_tables(df, database_name):
    conn = sqlite3.connect(database_name)
    cur = conn.cursor()

    for index, row in df.iterrows():
        # Insert data into Materials table
        material_id = index + 1  # assuming MaterialID starts from 1 and increments with each row
        cur.execute("""
            INSERT INTO Materials (MaterialID, Name, Density)
            VALUES (?, ?, ?)
        """, (material_id, row['Material'], row['Density']))

        # Retrieve elementID and weightFraction from 'Composition' and insert data into MaterialCompositions table
        for element_id, weight_fraction in row['Composition'].items():
            cur.execute("""
                INSERT INTO MaterialCompositions (MaterialID, ElementID, WeightFraction)
                VALUES (?, ?, ?)
            """, (material_id, element_id, weight_fraction))

    conn.commit()
    conn.close()


In [11]:
fill_tables(df, '/home/john/Documents/MCXrayTransport/data/data_sources/EPDL/EPDL.db')